In [1]:
import torch, torchvision

In [51]:
class CNN(torch.nn.Module):
  def __init__(self, hidden1_size: int, hidden_size2: int, output_size: int):
    super().__init__()
    
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

    self.flatten = torch.nn.Flatten()

    self.fc1 = torch.nn.Linear(in_features=32*7*7, out_features=hidden1_size)
    self.fc2 = torch.nn.Linear(in_features=hidden1_size, out_features=hidden_size2)
    
    self.ol = torch.nn.Linear(in_features=hidden_size2, out_features=output_size)
  
  def forward(self, input_data):
    max_pooled_conv1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(torch.nn.functional.relu(self.conv1(input_data)))
    max_pooled_conv2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)(torch.nn.functional.relu(self.conv2(max_pooled_conv1)))

    flattened = self.flatten(max_pooled_conv2)

    fc1_out = torch.nn.functional.relu(self.fc1(flattened))
    fc2_out = torch.nn.functional.relu(self.fc2(fc1_out))

    # outputs = torch.nn.functional.softmax(self.ol(fc2_out))

    return fc2_out
    
    

In [52]:
images = torch.randint(0, 255, (10000, 28, 28), dtype=torch.uint8)
labels = torch.randint(0, 9, (10000,), dtype=torch.long)

transormer = torchvision.transforms.Compose([torchvision.transforms.ConvertImageDtype(torch.float32),
                                             torchvision.transforms.Normalize((0.5,), (0.5,))])
transformed_data = transormer(images)

# wraping images and labels
input_data = torch.utils.data.TensorDataset(transformed_data, labels)

train_data, val_data = torch.utils.data.random_split(input_data, [8000, 2000])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, num_workers=1)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, num_workers=1)

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [53]:
model = CNN(hidden1_size=64, hidden_size2=64, output_size=10).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100

for epoch in range(epochs):
  train_loss = 0
  model.train()
  for i, (images, labels) in enumerate(train_loader):
    optimizer.zero_grad()
    images, labels = images.to(device), labels.to(device)
    outputs = model(images.unsqueeze(1))
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  
  model.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images.unsqueeze(1))
      loss = criterion(outputs, labels)
      val_loss += loss.item()
      preds = outputs.argmax(dim=1)
      correct += (preds == labels).sum().item()

  val_acc = correct / len(val_data)
  print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.4f}, "
				f"Val Loss={val_loss/len(val_loader):.4f}, Val Acc={val_acc:.4f}")


Epoch 1: Train Loss=3.0185, Val Loss=2.9978, Val Acc=0.1025
Epoch 2: Train Loss=2.9483, Val Loss=2.9975, Val Acc=0.1150
Epoch 3: Train Loss=2.9437, Val Loss=2.9982, Val Acc=0.1130
Epoch 4: Train Loss=2.9378, Val Loss=2.9990, Val Acc=0.1150
Epoch 5: Train Loss=2.9296, Val Loss=3.0013, Val Acc=0.1110
Epoch 6: Train Loss=2.9175, Val Loss=3.0055, Val Acc=0.1125
Epoch 7: Train Loss=2.8998, Val Loss=3.0146, Val Acc=0.1135
Epoch 8: Train Loss=2.8760, Val Loss=3.0276, Val Acc=0.1080
Epoch 9: Train Loss=2.8443, Val Loss=3.0445, Val Acc=0.1175
Epoch 10: Train Loss=2.8068, Val Loss=3.0628, Val Acc=0.1180
Epoch 11: Train Loss=2.7613, Val Loss=3.0920, Val Acc=0.1175
Epoch 12: Train Loss=2.7134, Val Loss=3.1256, Val Acc=0.1115
Epoch 13: Train Loss=2.6608, Val Loss=3.1646, Val Acc=0.1145
Epoch 14: Train Loss=2.6057, Val Loss=3.2192, Val Acc=0.1125
Epoch 15: Train Loss=2.5486, Val Loss=3.2723, Val Acc=0.1125
Epoch 16: Train Loss=2.4899, Val Loss=3.3278, Val Acc=0.1095
Epoch 17: Train Loss=2.4392, Val 